In [1]:
from my_trader import *

C:\ProgramData\Anaconda2\lib\site-packages\fix_yahoo_finance\__init__.py:43: DeprecationWarning: 
    Auto-overriding of pandas_datareader's get_data_yahoo() is deprecated and no longer available.
    Use pdr_override() to explicitly override it.
  DeprecationWarning)


In [2]:
robinhood = get_robinhood()
stock1 = "VIXY"
stock2 = "SPY"
window = 30
data_len = 210
initial = 1500


price1 = get_price_data([stock1],method="day",start_date=datetime.now()-timedelta(days=data_len),end_date=datetime.now()-timedelta(days=0))
price2 = get_price_data([stock2],method="day",start_date=datetime.now()-timedelta(days=data_len),end_date=datetime.now()-timedelta(days=0))
#price1 = get_price_data([stock1],method = "robinhood")
#price2 = get_price_data([stock2],method = "robinhood")
price1 = price1.set_index("TimeStamp")
price2 = price2.set_index("TimeStamp")
price1 = price1.rename(columns={"Close":stock1+"_close"})
price2 = price2.rename(columns={"Close":stock2+"_close"})



price_table = pd.concat([price1[stock1+"_close"],price2[stock2+"_close"]],axis = 1)
price_table = price_table.fillna(price_table.shift(1))
price_table[stock1+"_close"] = price_table[stock1+"_close"].astype(float)
price_table[stock2+"_close"] = price_table[stock2+"_close"].astype(float)
price_table[stock1+"_log_ret"] = log(price_table[stock1+"_close"] / price_table[stock1+"_close"].shift(1))
price_table[stock2+"_log_ret"] = log(price_table[stock2+"_close"] / price_table[stock2+"_close"].shift(1))

#price_table=price_table.fillna(1)

price_table[stock1+"_log_ret_mv"] = price_table[stock1+"_log_ret"].rolling(window).mean()
price_table[stock2+"_log_ret_mv"] =price_table[stock2+"_log_ret"].rolling(window).mean()
price_table["relative"]=price_table[stock1+"_log_ret"]/price_table[stock2+"_log_ret"]
price_table["relative_mv"] = price_table["relative"].rolling(window).mean()

price_table.relative.loc[price_table.relative==-np.inf]=np.NaN

#price_table["relative_mv"]=price_table[stock1+"_log_ret_mv"]/price_table[stock2+"_log_ret_mv"]
price_table["z_score"] =( price_table["relative"]-price_table["relative_mv"])/price_table.relative.std()
price_table["trade"]=0
price_table["trade_signal"]=np.NaN
price_table["slope"]=0
for i in range(window-1,len(price_table)):
    price_table["slope"].iloc[i] = stats.linregress(price_table[stock1+"_log_ret"]\
                                                    .iloc[i-(window-1):i],price_table[stock2+"_log_ret"].iloc[i-(window-1):i])[0]
price_table["buy_line"] = np.NaN



for i in range(window-1,len(price_table)):
    if price_table.slope.iloc[i]>0.5:
        price_table["buy_line"].iloc[i] = -1.25
    elif price_table.slope.iloc[i]>0.75:
        price_table["buy_line"].iloc[i]= -1.75
    elif price_table.slope.iloc[i]<-0.5:
        price_table["buy_line"].iloc[i] -2.25
    elif price_table.slope.iloc[i]<-0.75:
        price_table["buy_line"].iloc[i]= -2.75
    else:
        price_table["buy_line"].iloc[i] = -2
price_table["sell_line"] = np.NaN
for i in range(window-1,len(price_table)):
    if price_table.slope.iloc[i]>0.5:
        price_table["sell_line"].iloc[i] = 1.25
    elif price_table.slope.iloc[i]>0.75:
        price_table["sell_line"].iloc[i] = 1.75
    elif price_table.slope.iloc[i]<-0.5:
        price_table["sell_line"].iloc[i] = 2.25
    elif price_table.slope.iloc[i]<-0.75:
        price_table["sell_line"].iloc[i] = 2.75
    else:
        price_table["sell_line"].iloc[i] = 2  
           
#no short sell

price_table["relative_mv"] = abs(price_table["relative_mv"])

        
#price_table["cost_per_trade"] =abs( price_table[stock1+"_close"]+price_table[stock2+"_close"]*price_table["relative_mv"])
price_table[stock1+"_suggest_shares"] = np.ceil((initial/(1+price_table["relative_mv"])/price_table[stock1+"_close"]))
price_table[stock2+"_suggest_shares"] = np.ceil((initial/(1+price_table["relative_mv"])*price_table["relative_mv"]/price_table[stock2+"_close"]))



#set live trade signal and backtest 
price_table[stock1+"_shares"] = 0 
price_table[stock2+"_shares"] = 0
for i in range(window-1,len(price_table)):
    if price_table.z_score.iloc[i] < price_table.buy_line.iloc[i]:
        price_table["trade"].iloc[i] = 1
        price_table["trade_signal"].iloc[i] = 1
        price_table[stock1+"_shares"].iloc[i] = price_table[stock1+"_suggest_shares"].iloc[i]
        price_table[stock2+"_shares"].iloc[i] = price_table[stock2+"_suggest_shares"].iloc[i]
    elif price_table.z_score.iloc[i] > price_table.sell_line.iloc[i]:
        price_table["trade"].iloc[i] = 0
        price_table["trade_signal"].iloc[i] = 0
        price_table[stock1+"_shares"].iloc[i] = 0 
        price_table[stock2+"_shares"].iloc[i] = 0
    else:
        price_table["trade"].iloc[i] = price_table.trade.iloc[i-1]
        price_table[stock1+"_shares"].iloc[i] = price_table[stock1+"_shares"].iloc[i-1]
        price_table[stock2+"_shares"].iloc[i] = price_table[stock2+"_shares"].iloc[i-1]

#price_table[stock1+"_value"] = price_table[stock1+"_suggest_shares"]*price_table[stock1+"_close"]
#price_table[stock2+"_value"] = price_table[stock2+"_suggest_shares"]*price_table[stock2+"_close"]

price_table[stock1+"_value"] = price_table[stock1+"_shares"]*price_table[stock1+"_close"]
price_table[stock2+"_value"] = price_table[stock2+"_shares"]*price_table[stock2+"_close"]



price_table["p_L"] = price_table[stock1+"_value"].shift(1) * price_table[stock1+"_log_ret"].shift(1)*price_table.trade.shift(1) +  \
price_table[stock2+"_value"].shift(1) * price_table[stock2+"_log_ret"].shift(1)*price_table.trade.shift(1)
price_table["rolling_p_L"] = price_table.p_L.rolling(30).sum()

Finished VIXY
Finished SPY


C:\ProgramData\Anaconda2\lib\site-packages\pandas\core\indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [65]:



def pair_trade(stock1, stock2,initial,method = "day",window = 30,data_len = 210, continuous = False):    
    robinhood = get_robinhood()
    # stock1 = "VIXY"
    # stock2 = "SPY"
    if method == "day":
        price1 = get_price_data([stock1],method="day",start_date=datetime.now()-timedelta(days=data_len),end_date=datetime.now()-timedelta(days=0))
        price2 = get_price_data([stock2],method="day",start_date=datetime.now()-timedelta(days=data_len),end_date=datetime.now()-timedelta(days=0))
    elif method == "minute":
        price1 = get_price_data([stock1],method = "robinhood")
        price2 = get_price_data([stock2],method = "robinhood")
    price1 = price1.set_index("TimeStamp")
    price2 = price2.set_index("TimeStamp")
    price1 = price1.rename(columns={"Close":stock1+"_close"})
    price2 = price2.rename(columns={"Close":stock2+"_close"})



    price_table = pd.concat([price1[stock1+"_close"],price2[stock2+"_close"]],axis = 1)
    price_table = price_table.fillna(price_table.shift(1))
    price_table[stock1+"_close"] = price_table[stock1+"_close"].astype(float)
    price_table[stock2+"_close"] = price_table[stock2+"_close"].astype(float)
    price_table[stock1+"_log_ret"] = log(price_table[stock1+"_close"] / price_table[stock1+"_close"].shift(1))
    price_table[stock2+"_log_ret"] = log(price_table[stock2+"_close"] / price_table[stock2+"_close"].shift(1))

    #price_table=price_table.fillna(1)

    price_table[stock1+"_log_ret_mv"] = price_table[stock1+"_log_ret"].rolling(window).mean()
    price_table[stock2+"_log_ret_mv"] =price_table[stock2+"_log_ret"].rolling(window).mean()
    price_table["relative"]=price_table[stock1+"_log_ret"]/price_table[stock2+"_log_ret"]
    price_table["relative_mv"] = price_table["relative"].rolling(window).mean()

    price_table.relative.loc[price_table.relative==-np.inf]=price_table.relative.shift(1)
    price_table.relative.loc[price_table.relative==-np.NaN]=price_table.relative.shift(1)
    #price_table["relative_mv"]=price_table[stock1+"_log_ret_mv"]/price_table[stock2+"_log_ret_mv"]
    price_table["z_score"] =( price_table["relative"]-price_table["relative_mv"])/price_table.relative.std()
    price_table["trade"]=0
    price_table["trade_signal"]=np.NaN
    price_table["slope"]=0
    for i in range(window-1,len(price_table)):
        price_table["slope"].iloc[i] = stats.linregress(price_table[stock1+"_log_ret"]\
                                                        .iloc[i-(window-1):i],price_table[stock2+"_log_ret"].iloc[i-(window-1):i])[0]
    price_table["buy_line"] = np.NaN

    print ("Log return done, 10%")

    for i in range(window-1,len(price_table)):
        if price_table.slope.iloc[i]>0.5:
            price_table["buy_line"].iloc[i] = -1.25
        elif price_table.slope.iloc[i]>0.75:
            price_table["buy_line"].iloc[i]= -1.75
        elif price_table.slope.iloc[i]<-0.5:
            price_table["buy_line"].iloc[i] -2.25
        elif price_table.slope.iloc[i]<-0.75:
            price_table["buy_line"].iloc[i]= -2.75
        else:
            price_table["buy_line"].iloc[i] = -2
     
    price_table["sell_line"] = np.NaN
    for i in range(window-1,len(price_table)):
        if price_table.slope.iloc[i]>0.5:
            price_table["sell_line"].iloc[i] = 1.25
        elif price_table.slope.iloc[i]>0.75:
            price_table["sell_line"].iloc[i] = 1.75
        elif price_table.slope.iloc[i]<-0.5:
            price_table["sell_line"].iloc[i] = 2.25
        elif price_table.slope.iloc[i]<-0.75:
            price_table["sell_line"].iloc[i] = 2.75
        else:
            price_table["sell_line"].iloc[i] = 2  
    print ("Signal Line done, 30%")           
    #no short sell

    price_table["relative_mv"] = abs(price_table["relative_mv"])

            
    #price_table["cost_per_trade"] =abs( price_table[stock1+"_close"]+price_table[stock2+"_close"]*price_table["relative_mv"])
    price_table[stock1+"_suggest_shares"] = np.ceil((initial/(1+price_table["relative_mv"])/price_table[stock1+"_close"]))
    price_table[stock2+"_suggest_shares"] = np.ceil((initial/(1+price_table["relative_mv"])*price_table["relative_mv"]/price_table[stock2+"_close"]))


    #set live trade signal and backtest 
    price_table[stock1+"_shares"] = 0 
    price_table[stock2+"_shares"] = 0
    for i in range(window-1,len(price_table)):
        if price_table.z_score.iloc[i] < price_table.buy_line.iloc[i]:
            price_table["trade"].iloc[i] = 1
            price_table["trade_signal"].iloc[i] = 1
            price_table[stock1+"_shares"].iloc[i] = price_table[stock1+"_suggest_shares"].iloc[i]
            price_table[stock2+"_shares"].iloc[i] = price_table[stock2+"_suggest_shares"].iloc[i]
        elif price_table.z_score.iloc[i] > price_table.sell_line.iloc[i]:
            price_table["trade"].iloc[i] = 0
            price_table["trade_signal"].iloc[i] = 0
            price_table[stock1+"_shares"].iloc[i] = 0 
            price_table[stock2+"_shares"].iloc[i] = 0
        else:
            price_table["trade"].iloc[i] = price_table.trade.iloc[i-1]
            price_table[stock1+"_shares"].iloc[i] = price_table[stock1+"_shares"].iloc[i-1]
            price_table[stock2+"_shares"].iloc[i] = price_table[stock2+"_shares"].iloc[i-1]
    print ("Trade singal done, 60%") 
    if not continuous:
        price_table[stock1+"_value"] = price_table[stock1+"_shares"].shift(1)*price_table[stock1+"_close"].shift(1)
        price_table[stock2+"_value"] = price_table[stock2+"_shares"].shift(1)*price_table[stock2+"_close"].shift(1)
        price_table["p_L"] = price_table[stock1+"_value"].shift(1) * price_table[stock1+"_log_ret"].shift(1) +  \
        price_table[stock2+"_value"].shift(1) * price_table[stock2+"_log_ret"].shift(1)
    else:    
        price_table[stock1+"_value"] = price_table[stock1+"_suggest_shares"].shift(1)*price_table[stock1+"_close"].shift(1)
        price_table[stock2+"_value"] = price_table[stock2+"_suggest_shares"].shift(1)*price_table[stock2+"_close"].shift(1)
        price_table["p_L"] = price_table[stock1+"_value"].shift(1) * price_table[stock1+"_log_ret"].shift(1)*price_table.trade.shift(1) +  \
        price_table[stock2+"_value"].shift(1) * price_table[stock2+"_log_ret"].shift(1)*price_table.trade.shift(1)

    print ("Finalizing, 90%")
    price_table["rolling_p_L"] = price_table.p_L.rolling(window).sum()
    return price_table

In [72]:
price = pair_trade("VIXY","SPY",1500,continuous=True)

Finished VIXY
Finished SPY
Log return done, 10%
Signal Line done, 30%
Trade singal done, 60%
Finalizing, 90%


In [73]:
price

,VIXY_close,SPY_close,VIXY_log_ret,SPY_log_ret,VIXY_log_ret_mv,SPY_log_ret_mv,relative,relative_mv,z_score,trade,...,buy_line,sell_line,VIXY_suggest_shares,SPY_suggest_shares,VIXY_shares,SPY_shares,VIXY_value,SPY_value,p_L,rolling_p_L
TimeStamp,,,,,,,,,,,,,,,,,,,,,
2017-07-13,40.799999,241.976151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2017-07-14,40.000000,243.104752,-0.019803,0.004653,NaN,NaN,-4.255644,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2017-07-17,38.930000,243.075043,-0.027114,-0.000122,NaN,NaN,221.859052,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2017-07-18,38.459999,243.203735,-0.012146,0.000529,NaN,NaN,-22.948433,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2017-07-19,38.009998,244.520447,-0.011769,0.005399,NaN,NaN,-2.179766,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2017-07-20,37.660000,244.629349,-0.009251,0.000445,NaN,NaN,-20.775474,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2017-07-21,37.310001,244.411560,-0.009337,-0.000891,NaN,NaN,10.483147,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2017-07-24,36.650002,244.352158,-0.017848,-0.000243,NaN,NaN,73.426999,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2017-07-25,36.799999,244.946152,0.004084,0.002428,NaN,NaN,1.682220,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [38]:
len(price)

145

In [64]:
price.to_csv("price_table.csv")

In [74]:
price_get = price.iloc[-1]

In [75]:
price_get["trade"]

1.0

In [76]:
price_get

VIXY_close               42.160000
SPY_close               262.679993
VIXY_log_ret              0.134074
SPY_log_ret              -0.018818
VIXY_log_ret_mv           0.020156
SPY_log_ret_mv           -0.000567
relative                 -7.124611
relative_mv               3.828222
z_score                  -0.053718
trade                     1.000000
trade_signal                   NaN
slope                    -0.160068
buy_line                 -2.000000
sell_line                 2.000000
VIXY_suggest_shares       8.000000
SPY_suggest_shares        5.000000
VIXY_shares               2.000000
SPY_shares                6.000000
VIXY_value              331.829991
SPY_value              1338.350065
p_L                       5.037845
rolling_p_L             183.375655
Name: 2018-02-08 00:00:00, dtype: float64

In [ ]:
positions